In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!tar -zxvf ../input/cifar10-python/cifar-10-python.tar.gz

In [ ]:
def load_data():
    """Loads CIFAR10 dataset.
    Returns:
      Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    import os
    import sys
    from six.moves import cPickle
    import numpy as np
    
    def load_batch(fpath):
        with open(fpath, 'rb') as f:
            d = cPickle.load(f, encoding='bytes')  
        data = d[b'data']
        labels = d[b'labels']
        data = data.reshape(data.shape[0], 3, 32, 32)
        return data, labels
    
    path = 'cifar-10-batches-py'
    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train[(i - 1) * 10000:i * 10000, :, :, :],
         y_train[(i - 1) * 10000:i * 10000]) = load_batch(fpath)
    
    x_test, y_test = load_batch(os.path.join(path, 'test_batch'))

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)


In [ ]:
(x_train,y_train),(x_test,y_test) = load_data()

In [ ]:
y = y_train.reshape(1,-1)

In [ ]:
y_train = y[0]

In [ ]:
y = y_test.reshape(1,-1)
y_test = y[0]

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers


In [ ]:
x_train.shape

In [ ]:
model = Sequential()

input_shape = (32, 32, 3)
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)
x_train1 = x_train.astype('float32')
x_test1 = x_test.astype('float32')
x_train1 /= 255.0
x_test1 /= 255.0
y_train1.shape

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), input_shape = (32, 32, 3),padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3), padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), padding='same', activation = 'relu', kernel_regularizer = regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()


In [ ]:
datagen = ImageDataGenerator(rotation_range=15, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True)
datagen.fit(x_train1)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer= 'adam',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(datagen.flow(x_train1, y_train1, batch_size=64),
                    steps_per_epoch=x_train.shape[0] // 64,epochs=200,
                    validation_data=(x_test1,y_test1))

In [ ]:
y_pred = model.predict_classes(x_test1, verbose=1)
y_pred1 = ['a' for i in range(len(y_pred))]
y_pred

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        y_pred1[i] = 'airplane'
    elif y_pred[i] == 1:
        y_pred1[i] = 'automobile'
    elif y_pred[i] == 2:
        y_pred1[i] = 'bird'
    elif y_pred[i] == 3:
        y_pred1[i] = 'cat'
    elif y_pred[i] == 4:
        y_pred1[i] = 'deer'
    elif y_pred[i] == 5:
        y_pred1[i] = 'dog'
    elif y_pred[i] == 6:
        y_pred1[i] = 'frog'
    elif y_pred[i] == 7:
        y_pred1[i] = 'horse'
    elif y_pred[i] == 8:
        y_pred1[i] = 'ship'
    elif y_pred[i] == 9:
        y_pred1[i] = 'truck'
y_pred1

In [ ]:
np.savetxt('project6.csv',y_pred1,delimiter=",",fmt = "%s")